In [ ]:
import os

import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np
from scipy.special import roots_legendre

import pyccl as ccl

import SSLimPy.interface.config as cfg
from SSLimPy.interface import sslimpy
from SSLimPy.cosmology import cosmology
from SSLimPy.cosmology import halo_model
from SSLimPy.LIMsurvey import higher_order
from SSLimPy.utils.utils import linear_interpolate, bilinear_interpolate, scalarProduct, addVectors

In [ ]:
envkey = "OMP_NUM_THREADS"
# Set this environment variable to the number of available cores in your machine,
# to get a fast execution of the Einstein Boltzmann Solver
print("The value of {:s} is: ".format(envkey), os.environ.get(envkey))
os.environ[envkey] = str(12)
print("The value of {:s} is: ".format(envkey), os.environ.get(envkey))

Choose a reference cosmology

In [ ]:
cosmo_dict = {
    "h": 0.7,
    "ns": 0.96,
    "sigma8": 0.82,
    "Omegab": 0.05,
    "Omegam": 0.32,
    "mnu": 0.06,
}

halo_dict = {
    "hmf_model":"ST",
    "bias_model": "b1",
    "nR" : 256,
    "Rmin": 1e-3 * u.Mpc,
    "Rmax": 1e3 * u.Mpc,
    "bloating": "Mead20",
}

## Obtain Cosmology results from SSLimPy


In [ ]:
settings = {"code":"class",
            "do_RSD" : True,
            "nonlinearRSD" : True,
            "FoG_damp" : "ISTF_like",
            "halo_model_PS" : True,
            "Smooth_window" : False,
            "nk":100,
            "kmax": 50*u.Mpc**-1,
            "kmin": 1e-4*u.Mpc**-1,
            "Smooth_resolution": False,
            }

In [ ]:
sslimpy.sslimpy(settings_dict=settings)

In [ ]:
s_cosmo = cosmology.CosmoFunctions(cosmopars=cosmo_dict)
s_halo = halo_model.HaloModel(s_cosmo, halo_dict)

In [ ]:
k = s_halo.k
print(np.min(k), np.max(k))

## Obtain Cosmology results from pyCCL


In [ ]:
Omega_c = s_cosmo.Omega(0, "clustering") - 0.05
cosmo = ccl.Cosmology(Omega_c=Omega_c, Omega_b=0.05, h=0.7, sigma8=0.82, n_s=0.96)

In [ ]:
# Halo choices in SSLimPy

cM = ccl.halos.ConcentrationDiemer15()
nM = ccl.halos.MassFuncSheth99(mass_def=ccl.halos.MassDef200c, mass_def_strict=False)
bM = ccl.halos.HaloBiasSheth99(mass_def=ccl.halos.MassDef200c, mass_def_strict=False)
pM = ccl.halos.HaloProfileNFW(mass_def=ccl.halos.MassDef200c, concentration=cM, fourier_analytic=True)

In [ ]:
hmc = ccl.halos.HMCalculator(mass_function=nM, halo_bias=bM)

## compare SSLimPy and pyCCL 

In [ ]:
m_arr = np.geomspace(1.01E12,1E15,128)/cosmo['h']

hmf_SL = s_halo.halomassfunction(m_arr * u.Msun, 0)
hmd_fof = ccl.halos.MassDefFof
h_cc = ccl.halos.MassFuncSheth99(mass_def=hmd_fof)
hmf_CC = h_cc(cosmo, m_arr, 1)

In [ ]:
plt.semilogx(m_arr, m_arr**2 * np.log(10) * hmf_SL, label="SSLimPy")
plt.semilogx(m_arr, m_arr * hmf_CC, label="pyCCL")
plt.legend()
plt.xlabel(r'$M$ $[M_\odot]$', fontsize=14)
plt.ylabel(r'$M\,\frac{dn}{d\log_{10}M}\,[M_\odot\,{\rm Mpc}^{-3}]$',
           fontsize=14)

In [ ]:
hb_SL = s_halo._bias_function.ST99(m_arr * u.Msun, 0, 1.686)
b_cc = ccl.halos.HaloBiasSheth99()
hb_CC = b_cc(cosmo, m_arr, 1)

In [ ]:
plt.semilogx(m_arr, hb_SL, label="SSLimPy")
plt.semilogx(m_arr, hb_CC, label="pyCCL")
plt.legend()
plt.xlabel(r'$M$ $[M_\odot]$', fontsize=14)
plt.ylabel(r'$b_h(M)$', fontsize=14)

In [ ]:
z = np.linspace(0,5)
plt.plot(z, s_cosmo.growth_rate(1e-4 / u.Mpc, z))
plt.xlabel("$z$", fontsize=14)
plt.ylabel(r"$\alpha_\mathrm{eff}$", fontsize=14)

In [ ]:
R = s_halo.R
nu_of_R = 1.686/s_halo.sigmaR_of_z(R, z, tracer="clustering")

neff_nu05 = np.empty_like(z)
neff_nu1 = np.empty_like(z)
neff_nu2 = np.empty_like(z)
neff_nu4 = np.empty_like(z)

for iz, zi in enumerate(z):
    R_of_nu = linear_interpolate(nu_of_R[:,iz], R, np.array([0.5, 1, 2, 4]))*u.Mpc
    neff_nu05[iz] = s_halo.n_eff_of_z(R_of_nu[0], zi, tracer="clustering")
    neff_nu1[iz] = s_halo.n_eff_of_z(R_of_nu[1], zi, tracer="clustering")
    neff_nu2[iz] = s_halo.n_eff_of_z(R_of_nu[2], zi, tracer="clustering")
    neff_nu4[iz] = s_halo.n_eff_of_z(R_of_nu[3], zi, tracer="clustering")

plt.plot(z, neff_nu05, label=r"$\nu$=0.5")
plt.plot(z, neff_nu1, label=r"$\nu$=1")
plt.plot(z, neff_nu2, label=r"$\nu$=2")
plt.plot(z, neff_nu4, label=r"$\nu$=4")
plt.xlabel("$z$", fontsize=14)
plt.ylabel(r"$n_\mathrm{eff}$", fontsize=14)
plt.legend()

In [ ]:
conc_SL = s_halo.concentration(m_arr * u.Msun, 0)
c_cc = ccl.halos.ConcentrationDiemer15()
p2pt = ccl.halos.Profile2pt()
conc_CC = c_cc(cosmo, m_arr, 1)

In [ ]:
plt.semilogx(m_arr, conc_SL, label="SSLimPy")
plt.semilogx(m_arr*s_halo.hubble, conc_CC, label="pyCCL")
plt.legend()
plt.xlabel(r'$M$ $[M_\odot]$', fontsize=14)
plt.ylabel(r'$c(M)$', fontsize=14)
plt.grid(which="both")

In [ ]:
plt.loglog(m_arr * s_halo.hubble, s_halo.concentration(m_arr * u.Msun, 0), label="$z=0.$")
plt.loglog(m_arr * s_halo.hubble, s_halo.concentration(m_arr * u.Msun, 0.5), label="$z=0.5$")
plt.loglog(m_arr * s_halo.hubble, s_halo.concentration(m_arr * u.Msun, 1), label="$z=1.$")
plt.legend()
plt.xlabel(r'$M$ $[M_\odot\,h^{-1}]$', fontsize=14)
plt.ylabel(r'$c(M)$', fontsize=14)
plt.grid(which="both")

In [ ]:
pk_CC = ccl.halos.halomod_power_spectrum(cosmo, hmc, k.value, 1., pM)
pk_SL = s_halo.P_halo(k, 0)
pk_CC_l = ccl.halos.halomod_power_spectrum(cosmo, hmc, k.value, 0.5, pM)
pk_SL_l = s_halo.P_halo(k, 1)

In [ ]:
plt.loglog(k, pk_SL, label="SSLimPy")
plt.loglog(k, pk_CC, label="pyCCL")
plt.xlabel(r"$k$ [$\mathrm{Mpc}^{-1}$]", fontsize=14)
plt.ylabel(r"$P_\mathrm{hh}(k)$ [$\mathrm{Mpc}^{3}$]", fontsize=14)
plt.legend()

In [ ]:
k = s_halo.k

In [ ]:
pM.get_normalization(cosmo, 1)

In [ ]:
I0_1_SS = s_halo.Ibeta_1(k, 0, beta=0)
I0_1_CL = hmc.I_0_1(cosmo, k.value, 1, pM) / pM.get_normalization(cosmo, 1)

plt.loglog(k, I0_1_SS)
plt.loglog(k, I0_1_CL)

In [ ]:
I1_1_SS = s_halo.Ibeta_1(k, 0, beta="b1")
I1_1_CL = hmc.I_1_1(cosmo, k.value, 1, pM)/ pM.get_normalization(cosmo, 1)

plt.loglog(k, I1_1_SS)
plt.loglog(k, I1_1_CL)

In [ ]:
I0_2_SS = s_halo.Ibeta_2(k, 0, beta=0)
I0_2_CL = hmc.I_0_2(cosmo, k.value, 1, pM, prof_2pt=p2pt) /  pM.get_normalization(cosmo, 1)**2

In [ ]:
plt.loglog(k, np.diagonal(I0_2_SS, axis1=0, axis2=1))
plt.loglog(k, I0_2_CL)

In [ ]:
I1_3_SS = s_halo.Ibeta_3(k, 0, beta="b1")
I1_3_CL = hmc.I_1_3(cosmo, k.value, 1, prof=pM, prof_2pt=p2pt) /  pM.get_normalization(cosmo, 1)**3

In [ ]:
plt.imshow(I1_3_CL/I1_3_SS.value-1)
plt.colorbar()

In [ ]:
I0_4_SS = s_halo.Ibeta_4(k, 0, beta=0)
I0_4_CL = hmc.I_0_22(cosmo, k.value, 1, prof=pM, prof2=pM, prof12_2pt=p2pt) /  pM.get_normalization(cosmo, 1)**4

In [ ]:
plt.loglog(k, [I0_4_SS[ik, ik].value for ik in range(len(k))])
plt.loglog(k, [I0_4_CL[ik, ik] for ik in range(len(k))])

In [ ]:
linpk_SL = s_cosmo.matpow(k, 0, nonlinear=False, tracer="clustering")
linpk_CC = ccl.linear_matter_power(cosmo, k.value, 1)
plt.loglog(k, linpk_SL, label="SSLimPy")
plt.loglog(k, linpk_CC, label="pyCCL")
plt.legend()
plt.xlabel(r"$k\,[\mathrm{Mpc}^{-1}]$", fontsize=14)
plt.ylabel(r"$P(k)\,[\mathrm{Mpc}^{3}]$", fontsize=14)

In [ ]:
# plt.loglog(k, pk_SL, label="SSLimPy")
plt.loglog(k, (I0_1_SS/max(I0_1_SS))**2 * linpk_SL + np.diagonal(I0_2_SS, axis1=0, axis2=1), label="Norm SSLimPy")
plt.loglog(k, pk_CC, label="pyCCL")
plt.xlabel(r"$k$ [$\mathrm{Mpc}^{-1}$]", fontsize=14)
plt.ylabel(r"$P_\mathrm{hh}(k)$ [$\mathrm{Mpc}^{3}$]", fontsize=14)
plt.legend()

## Obtain PT matter Trispectrum 

In [ ]:
ks = len(k)
mus = 9
xi, wi = roots_legendre(mus)
mu = xi

I01 = np.ones((ks, mus))

In [ ]:
mT_SL = higher_order._integrate_4h(1, 0, 0, 0, 0, 0, 0, 0, xi, wi, k, linpk_SL.value, I01)

In [ ]:
Ts_SL = np.empty((ks, ks, mus))
for ik, ki in enumerate(k.value):
    for jk, kj in enumerate(k.value):
        for imu, mui in enumerate(mu):
            Ts_SL[ik, jk, imu] = higher_order.TrispectrumL0(1, 0, 0, 0, 0, 0, 0, 0,
                                                            ki, 1.0, 0.0,
                                                            ki, -1.0, np.pi,
                                                            kj, mui, 0.0,
                                                            k, linpk_SL.value)

Tk_SL = np.sum(Ts_SL * wi, axis=-1)

In [ ]:
plt.loglog(k, Tk_SL[:, 20], label=r"$k_2$ = 0.01 $\mathrm{Mpc}^{-1}$")
plt.loglog(k, Tk_SL[:, 50], label=r"$k_2$ = 0.25 $\mathrm{Mpc}^{-1}$")
plt.loglog(k, Tk_SL[:, 80], label=r"$k_2$ = 6.6 $\mathrm{Mpc}^{-1}$")

plt.loglog(k, mT_SL[:, 20, 0, 0], "--")
plt.loglog(k, mT_SL[:, 50, 0, 0], "--")
plt.loglog(k, mT_SL[:, 80, 0, 0], "--")

plt.legend()

plt.xlabel(r"$k_1$ [$\mathrm{Mpc}^{-1}$]", fontsize=14)
plt.ylabel(r"$\langle T\rangle(k_1, k_2)$ [$\mathrm{Mpc}^{9}$]", fontsize=14)



# plt.loglog(k, [-Tk_SL[ik, ik] for ik in range(ks)])
# plt.vlines([k[50].value,k[20].value, k[80].value], 0, 1e18, "k")

In [ ]:
# Tk_CC = ccl.halos.halomod_Tk3D_4h(cosmo, hmc, pM)
# F = Tk_CC(k.value, 1)
F = np.loadtxt("CCL_Trispectrum.txt")

In [ ]:
plt.loglog(k, F[:, 20])
plt.loglog(k, F[:, 50])
plt.loglog(k, F[:, 80])
# plt.loglog(k, Tk_SL[:, 20],"k--", label=r"$k_2$ = 0.01 $\mathrm{Mpc}^{-1}$")
# plt.loglog(k, Tk_SL[:, 50],"k--", label=r"$k_2$ = 0.25 $\mathrm{Mpc}^{-1}$")
# plt.loglog(k, Tk_SL[:, 80],"k--", label=r"$k_2$ = 6.6 $\mathrm{Mpc}^{-1}$")

plt.loglog(k, mT_SL[:, 20, 0, 0], "--")
plt.loglog(k, mT_SL[:, 50, 0, 0], "--")
plt.loglog(k, mT_SL[:, 80, 0, 0], "--")

In [ ]:
k_use = k
def get_kr_and_f2_CC(theta):
    cth = np.cos(theta)
    kk = k_use[None, :]
    kp = k_use[:, None]
    kr2 = kk ** 2 + kp ** 2 + 2 * kk * kp * cth
    kr = np.sqrt(kr2)

    f2 = 5./7. - 0.5 * (1 + kk ** 2 / kr2) * (1 + kp / kk * cth) + \
        2/7. * kk ** 2 / kr2 * (1 + kp / kk * cth)**2
    # When kr = 0:
    # k^2 / kr^2 (1 + k / kr cos) -> k^2/(2k^2 + 2k^2 cos)*(1 + cos) = 1/2
    # k^2 / kr^2 (1 + k / kr cos)^2 -> (1 + cos)/2 = 0
    f2[np.where(kr == 0)] = 13. / 28

    return kr, f2
test = np.array([get_kr_and_f2_CC(i) for i in np.linspace(-np.pi, np.pi)])
k_CC, f2_CC = test[:,0, :, :], test[:, 1, :, :]

In [ ]:
def get_kr_and_f2_SL(theta):
    kr = np.empty((*k_use.shape,*k_use.shape))
    f2 = np.empty((*k_use.shape,*k_use.shape))
    for ik1, k1i in enumerate(k_use):
        for ik2, k2i in enumerate(k_use):
            try:
                kr[ik1, ik2], mu12, _ = addVectors(k1i.value, 1.0, 0.0, k2i.value, np.cos(theta), 0.0)
            except:
                print(k1i.value, 1.0, 0.0, k2i.value, np.cos(theta), 0.0)
        
            if np.isclose(kr[ik1,ik2], 0.0):
                f2[ik1, ik2] = 13./28.
            else:
                f2[ik1, ik2] = higher_order.vF2(k1i.value, 1.0, 0.0, kr[ik1, ik2], -mu12, np.pi)
        # print(np.cos(theta), scalarProduct(k1i.value, 1.0, 0.0, k2i.value, np.cos(theta), 0.0)/k1i.value/k2i.value)

    return kr, f2
test = np.array([get_kr_and_f2_SL(i) for i in np.linspace(-np.pi, np.pi)])
k_SL, f2_SL = test[:,0, :, :], test[:, 1, :, :]

In [ ]:
from SSLimPy.utils.utils import addVectors

addVectors(35.97573724640009, 1.0, 0.0, 35.97573724640009, -1.0, 0.0)

In [ ]:
from numba import njit

@njit(fastmath=True)
def addVectors(
    k1,
    mu1,
    ph1,
    k2,
    mu2,
    ph2,
):
    k1pk2 = scalarProduct(k1, mu1, ph1, k2, mu2, ph2)
    radicant = k1**2 + 2 * k1pk2 + k2**2
    if np.isclose(np.sqrt(np.abs(radicant)), 0):
        k12 = 0
        mu12 = 0
        phi12 = 0
    else:
        k12 = np.sqrt(radicant)

        mu12 = (k1 * mu1 + k2 * mu2) / k12
        # Sometimes numerics dont care about triangle equations
        if mu12 < -1.0:
            mu12 = -1.0
        if mu12 > 1.0:
            mu12 = 1.0

        if np.isclose(np.abs(mu12), 1):
            phi12 = 0
        else:
            s1s = 1 - mu1**2
            if np.isclose(s1s, 0):
                s1s = 0
            s2s = 1 - mu2**2
            if np.isclose(s2s, 0):
                s2s = 0

            x = k1 * np.sqrt(s1s) * np.cos(ph1) + k2 * np.sqrt(s2s) * np.cos(ph2)
            y = k1 * np.sqrt(s1s) * np.sin(ph1) + k2 * np.sqrt(s2s) * np.sin(ph2)

            if np.isclose(x, 0):
                if np.sign(y) == 1:
                    phi12 = np.pi
                else:
                    phi12 = -np.pi
            else:
                phi12 = np.arctan2(y, x)

    return k12, mu12, phi12

addVectors(35.97573724640009, 1.0, 0.0, 35.97573724640009, -1.0, 0.0)

In [ ]:
import seaborn as sns
C = sns.color_palette("Paired")
C

In [ ]:
colors = iter(C)

theta = np.linspace(-np.pi, np.pi)
plt.plot(theta, k_SL[:, 15, 25],c=next(colors),ls="-")
plt.plot(theta, k_CC[:, 15, 25],c=next(colors),ls="--")
plt.plot(theta, k_SL[:, 25, 25],c=next(colors),ls="-")
plt.plot(theta, k_CC[:, 25, 25],c=next(colors),ls="--")
plt.xlabel(r"$\theta$")
plt.ylabel(r"$k_{12}$")

In [ ]:
colors = iter(C)

plt.plot(theta, f2_SL[:, 15, 25],c=next(colors),ls="-",label="SSLimPy")
plt.plot(theta, f2_CC[:, 25, 15],c=next(colors),ls="--",label="pyccl")
plt.plot(theta, f2_SL[:, 25, 25],c=next(colors),ls="-")
plt.plot(theta, f2_CC[:, 25, 25],c=next(colors),ls="--")
plt.legend()
plt.xlabel(r"$\theta$")
plt.ylabel(r"$F_{2}$")

In [ ]:
k_use = k
def get_mF3_CC():
    thetas = np.linspace(-np.pi, np.pi)
    F3 = np.empty((*k_use.shape, *k_use.shape, *thetas.shape))
    kk = k_use[None, :]
    kp = k_use[:, None]
    k = kk
    r = kp / k
    for i, theta in enumerate(thetas):
        cth = np.cos(theta)
        kr2 = k ** 2 + kp ** 2 + 2 * k * kp * cth
        kr = np.sqrt(kr2)

        f3 = (5 * r + (7 - 2*r**2)*cth) / (1 + r**2 + 2*r*cth) * \
                (3/7. * r + 0.5 * (1 + r**2) * cth + 4/7. * r * cth**2)
        f3[np.where(kr == 0)] = 0
        F3[:,:,i] = f3
    mF3 = -7/4 * (1 + r**2) + np.trapz(F3, thetas, axis=-1) / (2*np.pi)

    return 4/9 * mF3
mf3_CC = get_mF3_CC()

In [ ]:
def get_f3_SL(theta):
    f3 = np.empty((*k_use.shape,*k_use.shape))
    for ik1, k1i in enumerate(k_use):
        for ik2, k2i in enumerate(k_use):
            f3[ik1, ik2] = higher_order.vF3(k1i.value, 1.0, 0.0,
                                            k2i.value, np.cos(theta), 0,
                                            k2i.value, -np.cos(theta), np.pi)
            if np.isnan(f3[ik1, ik2]):
                print(theta, k1i, k2i)
    return f3

thetas = np.linspace(-np.pi, np.pi)

f3_SL = np.array([get_f3_SL(i) for i in thetas]) 
mf3_SL = np.trapz(f3_SL, thetas, axis=0)* 12 / (2 * np.pi)

In [ ]:
from matplotlib.lines import Line2D
colors = iter(C)

# Chosen indices for k1 values (modify labels if needed)
k1_indices = [15, 25, 65, 95]
k1_labels = [r"$k_1 = {:.2f}$".format(k_use[i]) for i in k1_indices]

# Plotting lines
for idx, k1 in zip(k1_indices, k1_labels):
    plt.loglog(k_use, -mf3_SL[:, idx], c=next(colors), ls="-", label=k1)
    plt.loglog(k_use, -mf3_CC[:, idx], c=next(colors), ls="--")

# Create custom legend for code line styles
line_styles = [
    Line2D([0], [0], color='black', lw=2, ls='-', label='SSLimPy'),
    Line2D([0], [0], color='black', lw=2, ls='--', label='pyccl')
]
legend1 = plt.legend(handles=line_styles, loc='lower left', title="Code")
plt.gca().add_artist(legend1)  # Add first legend manually

# Create legend for colors (k₁ values)
color_handles = []
colors = iter(C)
for i in range(len(k1_indices)):
    label = k1_labels[i]
    next(colors)
    color_handles.append(Line2D([0], [0], color=next(colors), lw=2, label=label))

plt.legend(handles=color_handles, loc='lower right', title=r"$k_1$ value")

plt.xlabel(r"$k_2 \, [\mathrm{Mpc}^{-1}]$")
plt.ylabel(r"$\langle F_3 \rangle \, [\mathrm{Mpc}^{-1}]$")
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.colors as mcolors

# Compute relative difference
rel_diff = (mf3_SL - mf3_CC) / mf3_CC
rel_diff = rel_diff.value

# Custom diverging colormap with out-of-bounds handling
cmap = plt.cm.magma  # good diverging colormapNone, :]
norm = mcolors.Normalize(vmin=-0.1, vmax=0.1)  # ±100%

# Create a copy with boundary colors
cmap = cmap.copy()
cmap.set_under('black')   # Color for < -100% difference
cmap.set_over('white')      # Color for > +100% difference

# Plot using imshow
plt.figure(figsize=(8, 6))
im = plt.imshow(rel_diff, cmap=cmap, norm=norm, origin='lower',
                extent=[k_use.value.min(), k_use.value.max(), k_use.value.min(), k_use.value.max()],
                aspect='auto')
plt.colorbar(im, label='Relative Difference (SSLimPy - pyccl) / pyccl',
             extend='both')  # show triangles for over/under colors
plt.xlabel(r"$k_2 \, [\mathrm{Mpc}^{-1}]$")
plt.ylabel(r"$k_1 \, [\mathrm{Mpc}^{-1}]$")
plt.title("Relative Difference Between Codes")
plt.tight_layout()
plt.show()